In [24]:
import requests
from bs4 import BeautifulSoup
import psycopg2
import time,re
from urllib.parse import urljoin

In [25]:
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="postgres",
    password="ja52712",
    port="5432"
)
print("Postgresに接続できた")



Postgresに接続できた


In [26]:
cursor = conn.cursor()

In [27]:
create_table_sql = """
create table if not exists users(
    id serial primary key,
    title text not null,
    product_url text unique not null
);
"""


conn.commit()

In [28]:
cursor.execute(create_table_sql)
conn.commit()
print("usersテーブル作成完了")

usersテーブル作成完了


In [29]:
cursor.execute("select * from users;")
rows = cursor.fetchall()
print(rows)

[(1, 'Test Book', 'https://books.toscrape.com/catalogue/test-book_1/index.html')]


In [30]:
url = "https://books.toscrape.com/"
items = []
page = 1

while url:
    res = requests.get(url,timeout=10)
    res.raise_for_status()
    soup = BeautifulSoup(res.text,"html.parser")
    
    for card in soup.select("article.product_pod"):
        title = card.h3.a["title"].strip()
        rel = card.h3.a["href"]
        abs_url = urljoin(url,rel)
        items.append((title,abs_url))
        
    next_link = soup.select_one("li.next > a")
    if next_link:
        url = urljoin(url,next_link["href"])
        page += 1
        time.sleep(0.5)
    else:
        url = None
        
print(f"\n取得完了：総ページ数{page},総アイテム数")

KeyboardInterrupt: 